# 055: Object Detection (YOLO, R-CNN)## 📚 Learning ObjectivesBy the end of this notebook, you will master:1. **Object Detection Fundamentals** - Bounding boxes, IoU, anchor boxes, multi-task learning2. **Two-Stage Detectors** - R-CNN evolution (R-CNN → Fast R-CNN → Faster R-CNN → Mask R-CNN)3. **Single-Stage Detectors** - YOLO architecture (YOLOv3, YOLOv5, YOLOv8), SSD, RetinaNet4. **Loss Functions** - Localization loss (L1, IoU, GIoU, DIoU), classification loss, focal loss5. **Non-Maximum Suppression (NMS)** - Post-processing, soft-NMS, DIoU-NMS6. **Evaluation Metrics** - mAP (mean Average Precision), IoU thresholds, COCO metrics7. **Real-Time Detection** - FPS optimization, model quantization, edge deployment8. **Semiconductor Applications** - PCB defect localization, die-level component detection, wafer map spatial analysis---## 🎯 Why Object Detection Matters### Classification vs Detection vs Segmentation```┌─────────────────────────────────────────────────────────────┐│ IMAGE UNDERSTANDING TASKS                                    │├─────────────────────────────────────────────────────────────┤│                                                              ││  ┌──────────────────┐  ┌──────────────────┐  ┌───────────┐ ││  │ CLASSIFICATION   │  │ DETECTION        │  │ SEGMENTATION││  ├──────────────────┤  ├──────────────────┤  ├───────────┤ ││  │ What?            │  │ What + Where?    │  │ Pixel-wise│ ││  │                  │  │                  │  │ masks     │ ││  │ [Dog]            │  │ [Dog] @ (x,y,w,h)│  │ [Outline] │ ││  │                  │  │ [Cat] @ (x,y,w,h)│  │ [Exact]   │ ││  │ Single label     │  │ Multiple objects │  │ Boundaries│ ││  │                  │  │ + Bounding boxes │  │ per pixel │ ││  └──────────────────┘  └──────────────────┘  └───────────┘ ││                                                              ││  Notebook 053          Notebook 055 (THIS)  Notebook 058   │└─────────────────────────────────────────────────────────────┘```**Object Detection = Classification + Localization**- **Input:** Image (H × W × 3)- **Output:**   - N bounding boxes: $(x, y, w, h)$ coordinates  - N class labels: $c \in \{1, 2, \ldots, C\}$  - N confidence scores: $p \in [0, 1]$---## 💼 Business Value for Semiconductor Industry### Use Case 1: PCB Defect Localization (vs Classification)**Previous approach (Notebook 053):** Classify entire image as "defective" or "normal"- **Problem:** Can't pinpoint defect location → Manual inspection still required**Object detection solution:** Detect and localize specific defects on PCB- **Output:** "Scratch @ (x=245, y=180, w=50, h=10), confidence=0.95"- **Business Impact:** $5M-$20M/year from automated defect localization- **Time Saved:** 80% reduction in manual inspection (from 30 sec/PCB → 6 sec)**Defect Types Detected:**1. Solder bridges (shorts between pins)2. Missing components (resistors, capacitors)3. Misaligned ICs4. Scratches on traces5. Contamination particles6. Cold solder joints---### Use Case 2: Die-Level Component Detection for Test Coverage**Problem:** Verify test probe placement on die (1000+ test points, 5mm × 5mm die)**Solution:** YOLOv8 detects all 1000+ probe pads in <50ms- **Input:** High-resolution die image (4096×4096 → 640×640 resize)- **Output:** Bounding boxes around each probe pad + classification (analog/digital/power)- **Business Impact:** $10M-$30M/year from reduced test escapes + faster test program validation- **Accuracy:** mAP@0.5 ≥ 0.98 (99%+ probe pads detected)---### Use Case 3: Wafer Map Spatial Defect Analysis**Previous approach (Notebooks 053-054):** Classify wafer map pattern (e.g., "center cluster")- **Limitation:** Can't identify multiple simultaneous defect clusters**Object detection solution:** Detect multiple defect regions on single wafer map- **Example:** "Cluster 1 @ center (30 dies), Cluster 2 @ edge (15 dies), Scratch @ (x=200, y=400)"- **Business Impact:** $20M-$80M/year from precise root-cause analysis- **Semiconductor benefit:** Correlate defect locations with process steps (lithography zones, etching patterns)---## 🏗️ What We'll Build### 1. **PCB Defect Detector (YOLOv8)**- **Task:** Detect 6 defect types on PCB images- **Dataset:** Synthetic PCB images with labeled defects (2000 images)- **Model:** YOLOv8-Medium (25M params, 40 FPS on GPU)- **Metrics:** mAP@0.5 ≥ 0.85, mAP@0.5:0.95 ≥ 0.60### 2. **Die Component Detector (Faster R-CNN)**- **Task:** Localize 50+ IC components on die photograph- **Approach:** Two-stage detection (region proposals → classification)- **Model:** Faster R-CNN with ResNet-50 backbone- **Metrics:** mAP@0.5 ≥ 0.92 (high precision needed for test validation)### 3. **Real-Time Inference Pipeline**- **Optimization:** TensorRT quantization (FP32 → INT8)- **Target:** 30 FPS on NVIDIA Jetson Nano (edge device)- **Deployment:** ONNX export → TensorRT engine → C++ inference---## 📊 Object Detection Architecture Evolution```mermaidgraph TD    A[Object Detection History] --> B[Two-Stage Detectors]    A --> C[Single-Stage Detectors]        B --> B1[R-CNN 2014<br/>Selective Search + CNN<br/>mAP: 53.3%<br/>Speed: 47 sec/image]    B1 --> B2[Fast R-CNN 2015<br/>RoI Pooling<br/>mAP: 66.9%<br/>Speed: 0.3 sec/image]    B2 --> B3[Faster R-CNN 2015<br/>Region Proposal Network<br/>mAP: 73.2%<br/>Speed: 0.2 sec/image]    B3 --> B4[Mask R-CNN 2017<br/>Instance Segmentation<br/>mAP: 37.1 mask<br/>Speed: 0.2 sec/image]        C --> C1[YOLO v1 2015<br/>Single pass<br/>mAP: 63.4%<br/>Speed: 45 FPS]    C1 --> C2[YOLOv3 2018<br/>Multi-scale<br/>mAP: 57.9%<br/>Speed: 30 FPS]    C2 --> C3[YOLOv5 2020<br/>PyTorch<br/>mAP: 66.3%<br/>Speed: 140 FPS]    C3 --> C4[YOLOv8 2023<br/>Anchor-free<br/>mAP: 71.8%<br/>Speed: 80 FPS]        B --> D{Trade-off}    C --> D    D --> E[Two-Stage: Higher mAP<br/>Slower inference<br/>Better small objects]    D --> F[Single-Stage: Real-time<br/>Lower mAP<br/>Better large objects]        style B1 fill:#ffe1e1    style B4 fill:#e1ffe1    style C1 fill:#ffe1e1    style C4 fill:#e1f5ff

```

---

## 🛠️ Notebook Structure

1. **Mathematical Foundations** - Bounding boxes, IoU, anchor boxes, loss functions
2. **Two-Stage Detection (Faster R-CNN)** - Region proposals, RoI pooling, implementation
3. **Single-Stage Detection (YOLOv8)** - Architecture, anchor-free detection, training
4. **Evaluation Metrics** - mAP calculation, COCO metrics, per-class analysis
5. **Real-Time Optimization** - TensorRT, quantization, FPS benchmarking
6. **Production Deployment** - 8 semiconductor + general AI/ML projects

---

## 📦 Prerequisites

**Libraries:**
```bash
# YOLOv8 (Ultralytics - state-of-the-art object detection)
pip install ultralytics

# Detectron2 (Facebook AI - Faster R-CNN, Mask R-CNN)
pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Core libraries
pip install torch torchvision opencv-python albumentations
pip install pycocotools  # COCO evaluation metrics
pip install onnx onnxruntime tensorrt  # Deployment

# Visualization
pip install matplotlib seaborn pillow
```

**Prior Knowledge:**
- Notebook 052: Deep Learning Frameworks (PyTorch basics)
- Notebook 053: CNN Architectures (convolution, ResNet)
- Notebook 054: Transfer Learning (fine-tuning pre-trained models)

---

## 📊 Dataset Overview

### Synthetic PCB Defect Dataset (6 Classes)

We'll generate **2000 PCB images** (640×640) with labeled defects:

| Class | Description | Avg. Instances/Image | Size Range (px) | Business Impact |
|-------|-------------|----------------------|-----------------|-----------------|
| 0 | Solder bridge | 0-3 | 10-30 | $50K-$200K/incident (short circuit) |
| 1 | Missing component | 0-2 | 20-50 | $100K-$500K (functionality loss) |
| 2 | Misaligned IC | 0-1 | 40-80 | $20K-$100K (assembly rework) |
| 3 | Scratch on trace | 0-4 | 15-60 | $10K-$50K (intermittent failure) |
| 4 | Contamination | 0-5 | 5-20 | $5K-$30K (yield loss) |
| 5 | Cold solder joint | 0-3 | 8-25 | $30K-$150K (reliability issue) |

**Annotation Format (YOLO):**
```
# Format: <class_id> <x_center> <y_center> <width> <height> (normalized 0-1)
0 0.523 0.341 0.045 0.032  # Solder bridge
1 0.720 0.615 0.038 0.051  # Missing component
3 0.102 0.890 0.095 0.015  # Scratch
```

**Key Characteristics:**
- **Multi-scale objects:** Small (contamination 5px) to large (IC 80px)
- **Class imbalance:** Missing components rare (0-2/image), contamination common (0-5/image)
- **Occlusion:** Components can overlap (e.g., scratch over IC)
- **Real-time requirement:** 30 FPS for production inspection

---

## 🎓 Learning Strategy

### Progressive Complexity
1. **Understand fundamentals:** IoU, anchor boxes, multi-task loss
2. **Implement two-stage:** Faster R-CNN (accuracy-focused)
3. **Implement single-stage:** YOLOv8 (speed-focused)
4. **Compare & optimize:** mAP vs FPS trade-off, deployment

### Experimentation Framework
For each detector, we'll measure:
- **mAP@0.5** (primary metric, IoU threshold = 0.5)
- **mAP@0.5:0.95** (COCO metric, average over IoU thresholds 0.5-0.95)
- **Inference FPS** (frames per second on GPU)
- **Model size** (parameters, disk size)
- **Per-class AP** (identify weak classes for improvement)

### Success Criteria
- **mAP@0.5:** ≥0.85 (85% of defects detected with IoU ≥ 0.5)
- **mAP@0.5:0.95:** ≥0.60 (robust across multiple IoU thresholds)
- **Inference speed:** ≥30 FPS on NVIDIA RTX 3080
- **Production deployment:** <50ms latency on Jetson Nano

---

## 🔗 How This Fits in the Learning Path

**Previous Notebooks:**
- 053: CNN Architectures → Image classification (what is in the image?)
- 054: Transfer Learning → Efficient training with pre-trained models

**This Notebook (055):**
- **Object detection** → Classification + Localization (what + where?)
- **Real-time inference** → Production-ready deployment
- **Multiple objects** → Detect 10-100 objects per image

**Next Notebooks:**
- 056: RNN/LSTM → Sequential data (test patterns over time)
- 057: Seq2Seq & Attention → Sequence-to-sequence learning
- 058: Semantic Segmentation → Pixel-wise classification

---

## 🚀 Let's Begin!

We'll start with mathematical foundations of object detection, then implement both two-stage (Faster R-CNN) and single-stage (YOLOv8) detectors.

---

# 📐 Part 1: Mathematical Foundations of Object Detection

## 🧮 Core Concepts

### 1. Bounding Box Representation

**Two formats:**

**Format 1: (x, y, w, h)** - YOLO format
- $(x, y)$: Center coordinates
- $(w, h)$: Width and height
- All values normalized to [0, 1]

**Format 2: (x₁, y₁, x₂, y₂)** - Pascal VOC format
- $(x_1, y_1)$: Top-left corner
- $(x_2, y_2)$: Bottom-right corner
- Absolute pixel coordinates

**Conversion:**
$$
\begin{aligned}
x_1 &= x - \frac{w}{2}, \quad y_1 = y - \frac{h}{2} \\
x_2 &= x + \frac{w}{2}, \quad y_2 = y + \frac{h}{2}
\end{aligned}
$$

**Example:**
```
YOLO format:  (0.5, 0.5, 0.2, 0.3) on 640×640 image
→ Center: (320, 320), Size: (128, 192)
→ Pascal VOC: (256, 224, 384, 416)
```

---

### 2. Intersection over Union (IoU)

**Definition:** Measure of overlap between predicted box $B_p$ and ground truth box $B_{gt}$

$$
\text{IoU}(B_p, B_{gt}) = \frac{\text{Area}(B_p \cap B_{gt})}{\text{Area}(B_p \cup B_{gt})} = \frac{\text{Intersection}}{\text{Union}}
$$

**Calculation Steps:**
1. Find intersection rectangle: 
   $$
   \begin{aligned}
   x_1^i &= \max(x_1^p, x_1^{gt}), \quad y_1^i = \max(y_1^p, y_1^{gt}) \\
   x_2^i &= \min(x_2^p, x_2^{gt}), \quad y_2^i = \min(y_2^p, y_2^{gt})
   \end{aligned}
   $$

2. Compute areas:
   $$
   \begin{aligned}
   \text{Area}_{\text{intersection}} &= \max(0, x_2^i - x_1^i) \times \max(0, y_2^i - y_1^i) \\
   \text{Area}_{\text{union}} &= \text{Area}_p + \text{Area}_{gt} - \text{Area}_{\text{intersection}}
   \end{aligned}
   $$

3. IoU:
   $$
   \text{IoU} = \frac{\text{Area}_{\text{intersection}}}{\text{Area}_{\text{union}}}
   $$

**IoU Interpretation:**
- **IoU ≥ 0.5:** Good detection (commonly used threshold)
- **IoU ≥ 0.7:** High-quality detection (strict evaluation)
- **IoU < 0.5:** False positive (bounding box too inaccurate)

**Example:**
```
Ground truth: (100, 100, 200, 200)  # 100×100 box
Prediction:   (120, 120, 220, 220)  # 100×100 box, shifted

Intersection: (120, 120, 200, 200) → 80×80 = 6400 px²
Union: 10000 + 10000 - 6400 = 13600 px²
IoU = 6400 / 13600 = 0.47 (FAILS at threshold 0.5)
```

---

### 3. Generalized IoU (GIoU) - Better Loss Function

**Problem with IoU loss:** When boxes don't overlap (IoU = 0), gradient is zero → No learning signal

**Solution: GIoU** - Considers smallest enclosing box

$$
\text{GIoU} = \text{IoU} - \frac{\text{Area}_C - \text{Area}_{\text{union}}}{\text{Area}_C}
$$

where $C$ is the smallest box enclosing both $B_p$ and $B_{gt}$.

**Properties:**
- **GIoU ∈ [-1, 1]** (IoU only [0, 1])
- **GIoU = IoU** when boxes overlap
- **GIoU < 0** when boxes don't overlap (provides gradient!)

**Loss function:**
$$
\mathcal{L}_{\text{GIoU}} = 1 - \text{GIoU}
$$

**Further improvements:**
- **DIoU (Distance IoU):** Penalizes distance between box centers
- **CIoU (Complete IoU):** Adds aspect ratio consistency

---

### 4. Anchor Boxes (YOLOv3-v5)

**Problem:** How to predict multiple objects of different sizes?

**Solution:** Pre-define anchor boxes at multiple scales

**Anchor box:** Template bounding box with specific aspect ratio and scale
- Small anchors: (10×10, 15×20, 20×15) for contamination, small defects
- Medium anchors: (30×30, 40×50, 50×40) for solder joints, scratches
- Large anchors: (60×60, 80×100, 100×80) for ICs, large components

**YOLOv3 uses 9 anchors (3 scales × 3 aspect ratios):**
```
Small:  (10,13), (16,30), (33,23)    # For 8×8 grid (stride 8)
Medium: (30,61), (62,45), (59,119)   # For 16×16 grid (stride 16)
Large:  (116,90), (156,198), (373,326) # For 32×32 grid (stride 32)
```

**Prediction:** Model predicts **offsets** from anchor box
$$
\begin{aligned}
x &= \sigma(t_x) + c_x  \quad \text{(c_x = grid cell x-coordinate)} \\
y &= \sigma(t_y) + c_y  \quad \text{(c_y = grid cell y-coordinate)} \\
w &= p_w \cdot e^{t_w}  \quad \text{(p_w = anchor width)} \\
h &= p_h \cdot e^{t_h}  \quad \text{(p_h = anchor height)}
\end{aligned}
$$

where $\sigma$ is sigmoid function, $t_x, t_y, t_w, t_h$ are model predictions.

---

### 5. Anchor-Free Detection (YOLOv8)

**Problem with anchors:** Requires careful tuning (k-means clustering on dataset), not universal

**YOLOv8 Solution:** Directly predict box coordinates (no anchors!)

**Architecture:**
- **Feature maps:** 3 scales (P3/8, P4/16, P5/32)
- **Output:** For each pixel $(i, j)$ on feature map:
  - Classification: $C$ classes (softmax)
  - Regression: $(x_1, y_1, x_2, y_2)$ bounding box (relative to pixel)
  - Objectness: Confidence score $p \in [0, 1]$

**Key innovation: Task Aligned Assigner (TAA)**
- Dynamically assigns ground truth boxes to feature map locations
- No fixed anchors → More flexible, better for diverse object sizes

---

### 6. Multi-Task Loss Function

**Object detection requires 3 simultaneous tasks:**

1. **Classification Loss** - What is the object?
   $$
   \mathcal{L}_{\text{cls}} = -\sum_{i \in \text{positive}} \log(\hat{p}_i^{c_i})
   $$
   where $\hat{p}_i^{c_i}$ is predicted probability for true class $c_i$.

2. **Localization Loss** - Where is the object?
   $$
   \mathcal{L}_{\text{loc}} = \sum_{i \in \text{positive}} \text{GIoU}(\hat{b}_i, b_i)
   $$
   where $\hat{b}_i$ is predicted box, $b_i$ is ground truth box.

3. **Objectness Loss** - Is there an object?
   $$
   \mathcal{L}_{\text{obj}} = -\sum_{i} \left[ o_i \log(\hat{o}_i) + (1 - o_i) \log(1 - \hat{o}_i) \right]
   $$
   where $o_i = 1$ if cell contains object center, 0 otherwise.

**Total Loss (YOLOv8):**
$$
\mathcal{L}_{\text{total}} = \lambda_{\text{cls}} \mathcal{L}_{\text{cls}} + \lambda_{\text{loc}} \mathcal{L}_{\text{loc}} + \lambda_{\text{obj}} \mathcal{L}_{\text{obj}}
$$

Typical weights: $\lambda_{\text{cls}} = 1.0$, $\lambda_{\text{loc}} = 5.0$, $\lambda_{\text{obj}} = 1.0$

---

### 7. Non-Maximum Suppression (NMS)

**Problem:** Model predicts many overlapping boxes for same object (e.g., 50 boxes around one IC)

**Solution: NMS** - Keep only the best box per object

**Algorithm:**
```
1. Sort all predictions by confidence score (descending)
2. While predictions remain:
   a. Take highest-confidence box B
   b. Add B to final detections
   c. Remove all boxes with IoU(box, B) > threshold (e.g., 0.45)
3. Return final detections
```

**Pseudocode:**
```python
def nms(boxes, scores, iou_threshold=0.45):
    # boxes: (N, 4), scores: (N,)
    keep = []
    sorted_indices = scores.argsort()[::-1]  # Descending order
    
    while len(sorted_indices) > 0:
        i = sorted_indices[0]
        keep.append(i)
        
        # Compute IoU with remaining boxes
        ious = compute_iou(boxes[i], boxes[sorted_indices[1:]])
        
        # Keep only boxes with IoU < threshold
        mask = ious < iou_threshold
        sorted_indices = sorted_indices[1:][mask]
    
    return boxes[keep], scores[keep]
```

**Variants:**
- **Soft-NMS:** Decay scores instead of removing (smooth suppression)
- **DIoU-NMS:** Use DIoU instead of IoU (considers box center distance)

---

## 📊 Evaluation Metrics: Mean Average Precision (mAP)

### Precision & Recall

**Precision:** Of all predicted boxes, how many are correct?
$$
\text{Precision} = \frac{TP}{TP + FP} = \frac{\text{Correct detections}}{\text{All detections}}
$$

**Recall:** Of all ground truth objects, how many did we find?
$$
\text{Recall} = \frac{TP}{TP + FN} = \frac{\text{Correct detections}}{\text{All ground truth objects}}
$$

**True Positive (TP):** Predicted box with IoU ≥ threshold (e.g., 0.5)  
**False Positive (FP):** Predicted box with IoU < threshold OR duplicate  
**False Negative (FN):** Ground truth object not detected

---

### Average Precision (AP) - Per Class

**Steps:**
1. Sort predictions by confidence (descending)
2. For each prediction, compute precision and recall
3. Plot Precision-Recall curve
4. **AP = Area under P-R curve**

**Example (Class: Solder bridge):**
```
Ground truth: 10 solder bridges in test set

Predictions (sorted by confidence):
1. Confidence=0.95, IoU=0.82 → TP → Precision=1/1=1.00, Recall=1/10=0.10
2. Confidence=0.92, IoU=0.76 → TP → Precision=2/2=1.00, Recall=2/10=0.20
3. Confidence=0.88, IoU=0.35 → FP → Precision=2/3=0.67, Recall=2/10=0.20
4. Confidence=0.85, IoU=0.91 → TP → Precision=3/4=0.75, Recall=3/10=0.30
...

AP = Integral of P-R curve ≈ 0.87
```

---

### Mean Average Precision (mAP)

**mAP@0.5:** Average AP across all classes at IoU threshold = 0.5
$$
\text{mAP@0.5} = \frac{1}{C} \sum_{c=1}^{C} \text{AP}_c^{0.5}
$$

**mAP@0.5:0.95 (COCO metric):** Average mAP over IoU thresholds [0.5, 0.55, 0.60, ..., 0.95]
$$
\text{mAP@0.5:0.95} = \frac{1}{10} \sum_{t=0.5}^{0.95} \text{mAP}^t
$$

**Interpretation:**
- **mAP@0.5 = 0.85:** 85% of objects detected with "good" localization
- **mAP@0.5:0.95 = 0.60:** Robust detection across multiple IoU thresholds (harder metric)

**COCO Metrics (Standard Benchmark):**
- **mAP:** mAP@0.5:0.95 (primary metric)
- **mAP@0.5:** Lenient (accepts rough boxes)
- **mAP@0.75:** Strict (requires tight boxes)
- **mAP_small:** AP for small objects (area < 32²)
- **mAP_medium:** AP for medium objects (32² < area < 96²)
- **mAP_large:** AP for large objects (area > 96²)

---

## 🎯 Two-Stage vs Single-Stage Detectors

### Two-Stage Detectors (Faster R-CNN)

**Stage 1: Region Proposal Network (RPN)**
- Generate ~2000 candidate object locations (region proposals)
- Use anchor boxes at multiple scales/aspect ratios
- Binary classification: Object vs Background
- Bounding box regression: Refine anchor positions

**Stage 2: RoI Classifier**
- Extract features from each region proposal (RoI Pooling)
- Multi-class classification: Which object class?
- Bounding box regression: Further refine box coordinates

**Pros:**
- ✅ **Higher mAP** (more accurate, especially for small objects)
- ✅ **Better localization** (two refinement stages)
- ✅ **State-of-the-art on COCO** (Mask R-CNN: mAP 37-40%)

**Cons:**
- ❌ **Slower** (10-20 FPS on GPU, two forward passes)
- ❌ **More complex** (two-stage training, harder to optimize)
- ❌ **Not real-time** (unsuitable for video processing)

---

### Single-Stage Detectors (YOLO, SSD, RetinaNet)

**Single forward pass:**
- Divide image into grid (e.g., 20×20 for YOLOv3)
- Each grid cell predicts: Bounding boxes + Classes + Confidence
- No region proposals (direct prediction)

**Pros:**
- ✅ **Real-time** (30-140 FPS on GPU)
- ✅ **Simple architecture** (end-to-end training)
- ✅ **Edge deployment** (YOLO runs on Jetson Nano, mobile devices)

**Cons:**
- ❌ **Lower mAP** (5-10% behind two-stage on small objects)
- ❌ **Struggles with small/dense objects** (grid-based prediction limits)

---

### Comparison Table

| Metric | Faster R-CNN | YOLOv8-Medium | YOLOv8-Nano |
|--------|--------------|---------------|-------------|
| **mAP@0.5 (COCO)** | 42.0% | 50.2% | 37.3% |
| **mAP@0.5:0.95 (COCO)** | 21.9% | 37.4% | 25.0% |
| **Inference (GPU)** | 15 FPS | 80 FPS | 180 FPS |
| **Parameters** | 137M | 26M | 3M |
| **Model Size** | 520 MB | 50 MB | 6 MB |
| **Small objects** | ✅ Excellent | ✅ Good | ⚠️ Fair |
| **Real-time** | ❌ No | ✅ Yes | ✅ Yes |

**Recommendation for Semiconductor:**
- **PCB defect detection (real-time):** YOLOv8-Medium (best balance)
- **Die-level inspection (accuracy):** Faster R-CNN (small components need precision)
- **Edge deployment (Jetson Nano):** YOLOv8-Nano (3M params, 180 FPS)

---

## 🔬 Next Steps

Now that we understand the theory, let's implement:
1. **YOLOv8** for real-time PCB defect detection
2. **Faster R-CNN** for high-precision die component localization
3. **Performance comparison** - mAP, FPS, deployment trade-offs

# 🚀 Part 2: YOLOv8 Implementation - PCB Defect Detection

## 📝 What's Happening in This Code?

**Purpose:** Train YOLOv8-Medium to detect 6 defect types on PCB images in real-time (80 FPS).

**Key Points:**
- **Ultralytics YOLOv8:** Latest YOLO version (2023), anchor-free, state-of-the-art
- **Dataset:** 2000 synthetic PCB images with 6 defect classes
- **Training:** Transfer learning from COCO pre-trained weights
- **Metrics:** mAP@0.5, mAP@0.5:0.95, per-class AP, confusion matrix

**YOLOv8 Architecture:**
```
Input (640×640×3)
    ↓
Backbone: CSPDarknet (Modified from YOLOv5)
    ├─ Conv + C2f blocks (C2f = improved C3 with skip connections)
    ├─ SPPF (Spatial Pyramid Pooling - Fast)
    ↓
Neck: PANet (Path Aggregation Network)
    ├─ Bottom-up + Top-down feature fusion
    ├─ 3 detection heads (P3/8, P4/16, P5/32 scales)
    ↓
Head: Anchor-free decoupled head
    ├─ Classification branch (6 defect classes)
    ├─ Regression branch (bounding box)
    ├─ Objectness branch (confidence)
    ↓
Output: N detections × (x, y, w, h, confidence, class_prob[6])
    ↓
NMS (Non-Maximum Suppression)
    ↓
Final Detections
```

**Why YOLOv8 for Semiconductor:**
- **Real-time:** 80 FPS on RTX 3080 (production line speed)
- **Accurate:** mAP@0.5 ≈ 85-90% (industry requirement: >80%)
- **Deployable:** ONNX/TensorRT export for Jetson Nano
- **Small model size:** 50 MB (YOLOv8-Medium) fits on edge devices

---

## 🔧 Implementation: YOLOv8 PCB Defect Detector

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
# ========================================
# Part 2: YOLOv8 Implementation
# ========================================
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import time
from ultralytics import YOLO
from PIL import Image, ImageDraw
import warnings
warnings.filterwarnings('ignore')
# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
# ========================================
# Generate Synthetic PCB Dataset
# ========================================
print("\n" + "="*70)
print("GENERATING SYNTHETIC PCB DEFECT DATASET")
print("="*70)
# Create dataset directory structure
dataset_root = Path("pcb_defect_dataset")
(dataset_root / "images" / "train").mkdir(parents=True, exist_ok=True)
(dataset_root / "images" / "val").mkdir(parents=True, exist_ok=True)
(dataset_root / "images" / "test").mkdir(parents=True, exist_ok=True)
(dataset_root / "labels" / "train").mkdir(parents=True, exist_ok=True)
(dataset_root / "labels" / "val").mkdir(parents=True, exist_ok=True)
(dataset_root / "labels" / "test").mkdir(parents=True, exist_ok=True)
# Defect classes
classes = [
    "solder_bridge",      # 0
    "missing_component",  # 1
    "misaligned_ic",      # 2
    "scratch",            # 3
    "contamination",      # 4
    "cold_solder"         # 5
]
def generate_pcb_image(img_size=640, num_defects_range=(1, 8)):
    """Generate synthetic PCB image with defects."""
    # Create base PCB (green background with copper traces)
    img = np.ones((img_size, img_size, 3), dtype=np.uint8) * np.array([34, 139, 34])  # Green PCB
    
    # Add copper traces (brown lines)
    for _ in range(20):
        if np.random.rand() > 0.5:
            # Horizontal trace
            y = np.random.randint(0, img_size)
            thickness = np.random.randint(3, 8)
            cv2.line(img, (0, y), (img_size, y), (139, 69, 19), thickness)
        else:
            # Vertical trace
            x = np.random.randint(0, img_size)
            thickness = np.random.randint(3, 8)
            cv2.line(img, (x, 0), (x, img_size), (139, 69, 19), thickness)
    
    # Add components (gray/black rectangles)
    for _ in range(np.random.randint(30, 50)):
        x = np.random.randint(20, img_size - 60)
        y = np.random.randint(20, img_size - 60)
        w = np.random.randint(15, 50)
        h = np.random.randint(10, 40)
        color = tuple(np.random.randint(40, 80, 3).tolist())
        cv2.rectangle(img, (x, y), (x+w, y+h), color, -1)
    
    # Add noise
    noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
    img = cv2.add(img, noise)
    
    # Generate defects
    num_defects = np.random.randint(*num_defects_range)
    annotations = []  # List of (class_id, x_center, y_center, width, height) normalized
    
    for _ in range(num_defects):
        defect_class = np.random.choice(range(len(classes)), p=[0.15, 0.10, 0.08, 0.25, 0.30, 0.12])
        
        if defect_class == 0:  # Solder bridge
            x = np.random.randint(50, img_size - 50)
            y = np.random.randint(50, img_size - 50)
            w, h = np.random.randint(10, 30), np.random.randint(5, 15)
            cv2.ellipse(img, (x, y), (w//2, h//2), 0, 0, 360, (200, 200, 200), -1)
        
        elif defect_class == 1:  # Missing component
            x = np.random.randint(50, img_size - 100)
            y = np.random.randint(50, img_size - 80)
            w, h = np.random.randint(25, 55), np.random.randint(20, 50)
            # Draw empty space (show PCB background)
            cv2.rectangle(img, (x, y), (x+w, y+h), (34, 139, 34), -1)
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), 2)  # Outline
        
        elif defect_class == 2:  # Misaligned IC
            x = np.random.randint(50, img_size - 100)
            y = np.random.randint(50, img_size - 100)
            w, h = np.random.randint(40, 80), np.random.randint(40, 80)
            # Draw rotated rectangle (misaligned)
            angle = np.random.randint(5, 20)
            rect = ((x+w//2, y+h//2), (w, h), angle)
            box = cv2.boxPoints(rect).astype(int)
            cv2.fillPoly(img, [box], (60, 60, 60))
        
        elif defect_class == 3:  # Scratch
            x1 = np.random.randint(0, img_size - 100)
            y1 = np.random.randint(0, img_size - 20)
            length = np.random.randint(50, 150)
            angle = np.random.uniform(-30, 30)
            x2 = int(x1 + length * np.cos(np.radians(angle)))
            y2 = int(y1 + length * np.sin(np.radians(angle)))
            thickness = np.random.randint(2, 5)
            cv2.line(img, (x1, y1), (x2, y2), (180, 180, 180), thickness)
            # Bounding box for annotation
            x = min(x1, x2)
            y = min(y1, y2)
            w = abs(x2 - x1) + 10
            h = abs(y2 - y1) + 10
        
        elif defect_class == 4:  # Contamination
            x = np.random.randint(10, img_size - 30)
            y = np.random.randint(10, img_size - 30)
            w, h = np.random.randint(5, 20), np.random.randint(5, 20)
            color = tuple(np.random.randint(150, 255, 3).tolist())
            cv2.circle(img, (x, y), w//2, color, -1)
        
        elif defect_class == 5:  # Cold solder joint
            x = np.random.randint(30, img_size - 30)
            y = np.random.randint(30, img_size - 30)
            w, h = np.random.randint(8, 25), np.random.randint(8, 25)
            # Draw irregular blob (cold solder)
            points = []
            for angle in range(0, 360, 30):
                r = np.random.randint(w//2 - 3, w//2 + 3)
                px = int(x + r * np.cos(np.radians(angle)))
                py = int(y + r * np.sin(np.radians(angle)))
                points.append([px, py])
            cv2.fillPoly(img, [np.array(points)], (150, 150, 150))
        
        # Compute normalized YOLO annotation (x_center, y_center, width, height)
        x_center = (x + w/2) / img_size
        y_center = (y + h/2) / img_size
        width_norm = w / img_size
        height_norm = h / img_size
        
        # Clip to [0, 1] and ensure valid box
        x_center = np.clip(x_center, 0, 1)
        y_center = np.clip(y_center, 0, 1)
        width_norm = np.clip(width_norm, 0.01, 1)
        height_norm = np.clip(height_norm, 0.01, 1)
        
        annotations.append((defect_class, x_center, y_center, width_norm, height_norm))
    
    return img, annotations
# Generate dataset
num_train = 1400
num_val = 400
num_test = 200
total = num_train + num_val + num_test
print(f"\nGenerating {total} PCB images...")
print(f"  Train: {num_train}")
print(f"  Val:   {num_val}")
print(f"  Test:  {num_test}")
start_time = time.time()
for split, num_images in [("train", num_train), ("val", num_val), ("test", num_test)]:
    for i in range(num_images):
        img, annotations = generate_pcb_image()
        
        # Save image
        img_path = dataset_root / "images" / split / f"pcb_{split}_{i:04d}.jpg"
        cv2.imwrite(str(img_path), img)
        
        # Save labels (YOLO format)
        label_path = dataset_root / "labels" / split / f"pcb_{split}_{i:04d}.txt"
        with open(label_path, 'w') as f:
            for ann in annotations:
                class_id, x_c, y_c, w, h = ann
                f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
    
    if (split == "train" and i % 500 == 0) or split != "train":
        print(f"  Generated {split}: {i+1}/{num_images}")
generation_time = time.time() - start_time
print(f"\n✓ Dataset generated in {generation_time:.2f} seconds")


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Create YAML Configuration
# ========================================
yaml_config = {
    'path': str(dataset_root.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(classes),
    'names': classes
}
yaml_path = dataset_root / "data.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_config, f, sort_keys=False)
print(f"\n✓ Created dataset configuration: {yaml_path}")
# ========================================
# Visualize Sample Images
# ========================================
print("\nVisualizing sample PCB images with defects...")
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for idx in range(6):
    img_path = dataset_root / "images" / "train" / f"pcb_train_{idx:04d}.jpg"
    label_path = dataset_root / "labels" / "train" / f"pcb_train_{idx:04d}.txt"
    
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img_rgb.shape[:2]
    
    # Read annotations
    with open(label_path, 'r') as f:
        annotations = [line.strip().split() for line in f.readlines()]
    
    # Draw bounding boxes
    for ann in annotations:
        class_id, x_c, y_c, width, height = map(float, ann)
        class_id = int(class_id)
        
        # Convert to pixel coordinates
        x_c_px, y_c_px = int(x_c * w), int(y_c * h)
        w_px, h_px = int(width * w), int(height * h)
        x1 = int(x_c_px - w_px/2)
        y1 = int(y_c_px - h_px/2)
        x2 = int(x_c_px + w_px/2)
        y2 = int(y_c_px + h_px/2)
        
        # Draw box
        color = plt.cm.tab10(class_id)[:3]
        color_255 = tuple(int(c*255) for c in color)
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color_255, 2)
        cv2.putText(img_rgb, classes[class_id], (x1, y1-5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, color_255, 1)
    
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'PCB Sample {idx+1}\n({len(annotations)} defects)', fontsize=10)
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig('pcb_samples.png', dpi=150, bbox_inches='tight')
print("✓ Saved visualization to 'pcb_samples.png'")
plt.show()
# ========================================
# Train YOLOv8


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
print("\n" + "="*70)
print("TRAINING YOLOv8-MEDIUM ON PCB DEFECT DATASET")
print("="*70)
# Load pre-trained YOLOv8-Medium
model = YOLO('yolov8m.pt')  # Medium model (26M parameters)
print(f"\nModel: YOLOv8-Medium")
print(f"  Parameters: ~26M")
print(f"  Pre-trained: COCO dataset")
# Training configuration
train_config = {
    'data': str(yaml_path),
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'device': device,
    'workers': 4,
    'patience': 10,  # Early stopping
    'save': True,
    'project': 'runs/detect',
    'name': 'pcb_yolov8m',
    'exist_ok': True,
    'verbose': True
}
print(f"\nTraining configuration:")
for k, v in train_config.items():
    print(f"  {k}: {v}")
# Train model
print(f"\nStarting training...")
start_time = time.time()
results = model.train(**train_config)
training_time = time.time() - start_time
print(f"\n✓ Training completed in {training_time/60:.2f} minutes")
# ========================================
# Evaluate Model
# ========================================
print("\n" + "="*70)
print("EVALUATING YOLOv8 ON TEST SET")
print("="*70)
# Load best weights
best_model = YOLO('runs/detect/pcb_yolov8m/weights/best.pt')
# Evaluate on test set
metrics = best_model.val(data=str(yaml_path), split='test')
print(f"\nTest Set Metrics:")
print(f"  mAP@0.5:       {metrics.box.map50:.4f}")
print(f"  mAP@0.5:0.95:  {metrics.box.map:.4f}")
print(f"  Precision:     {metrics.box.mp:.4f}")
print(f"  Recall:        {metrics.box.mr:.4f}")
# Per-class AP
print(f"\nPer-Class Average Precision (AP@0.5):")
for idx, class_name in enumerate(classes):
    ap = metrics.box.ap50[idx]
    print(f"  {class_name:20s}: {ap:.4f}")
# ========================================
# Inference Speed Benchmark
# ========================================
print("\n" + "="*70)
print("INFERENCE SPEED BENCHMARK")
print("="*70)
# Load test image
test_img_path = dataset_root / "images" / "test" / "pcb_test_0000.jpg"
test_img = cv2.imread(str(test_img_path))
# Warm-up
for _ in range(10):
    _ = best_model(test_img, verbose=False)
# Benchmark
num_runs = 100
start_time = time.time()
for _ in range(num_runs):
    results = best_model(test_img, verbose=False)
inference_time = (time.time() - start_time) / num_runs
fps = 1 / inference_time
print(f"\nInference Benchmarks ({num_runs} runs):")
print(f"  Average latency:  {inference_time*1000:.2f} ms")
print(f"  Throughput (FPS): {fps:.2f}")
print(f"  Device:           {device}")


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Visualize Predictions
# ========================================
print("\nVisualizing predictions on test images...")
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for idx in range(6):
    test_img_path = dataset_root / "images" / "test" / f"pcb_test_{idx:04d}.jpg"
    img = cv2.imread(str(test_img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = best_model(img, verbose=False)[0]
    
    # Draw predictions
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        
        if conf > 0.5:  # Confidence threshold
            color = plt.cm.tab10(cls)[:3]
            color_255 = tuple(int(c*255) for c in color)
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color_255, 2)
            label = f"{classes[cls]} {conf:.2f}"
            cv2.putText(img_rgb, label, (x1, y1-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color_255, 1)
    
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'Test Image {idx+1}\n({len(results.boxes)} detections)', fontsize=10)
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig('pcb_predictions.png', dpi=150, bbox_inches='tight')
print("✓ Saved predictions to 'pcb_predictions.png'")
plt.show()
# ========================================
# Export to ONNX (for deployment)
# ========================================
print("\n" + "="*70)
print("EXPORTING MODEL TO ONNX")
print("="*70)
onnx_path = best_model.export(format='onnx', simplify=True)
print(f"\n✓ Exported to ONNX: {onnx_path}")
print(f"  Compatible with: TensorRT, ONNX Runtime, OpenVINO")
print(f"  Deployment targets: Jetson Nano, edge devices, production servers")
print("\n" + "="*70)
print("YOLOV8 TRAINING & EVALUATION COMPLETE")
print("="*70)


# 🎯 Part 3: Faster R-CNN Comparison & Real-World Projects

## 📝 Faster R-CNN Overview

**Two-Stage Architecture:**

```
Stage 1: Region Proposal Network (RPN)
    Input (640×640×3)
        ↓
    Backbone: ResNet-50
        ↓
    Feature Map (20×20×2048)
        ↓
    Anchor Generation (9 anchors/cell × 400 cells = 3600 proposals)
        ├─ 3 scales: 128², 256², 512²
        └─ 3 aspect ratios: 1:1, 1:2, 2:1
        ↓
    Binary Classification: Object vs Background
    Bounding Box Regression: Refine anchor positions
        ↓
    Top-N proposals (e.g., N=300) sent to Stage 2

Stage 2: RoI Head
    For each proposal:
        ↓
    RoI Pooling (extract 7×7 features)
        ↓
    Fully Connected Layers
        ├─ Multi-class Classification (6 classes)
        └─ Bounding Box Refinement
        ↓
    NMS (remove duplicates)
        ↓
    Final Detections
```

**Key Differences: Faster R-CNN vs YOLOv8:**

| Feature | Faster R-CNN | YOLOv8 |
|---------|--------------|--------|
| **Stages** | Two-stage (RPN + RoI Head) | Single-stage (direct prediction) |
| **Anchors** | Fixed anchors (9 per cell) | Anchor-free |
| **Speed** | 10-20 FPS | 80-180 FPS |
| **mAP** | Higher (+3-5% on small objects) | Good (but 3-5% lower) |
| **Small objects** | ✅ Excellent (RPN captures small regions) | ⚠️ Fair (grid-based limits) |
| **Complexity** | High (two-stage training) | Low (end-to-end) |
| **Use case** | High-precision (die components, medical) | Real-time (PCB inspection, video) |

---

## 🚀 Real-World Projects

### **🔬 Semiconductor Projects (Post-Silicon Validation)**

#### **Project 1: Production PCB Defect Inspector with Real-Time Feedback**

**Objective:** Deploy YOLOv8 on factory production line for 100% automated visual inspection

**Business Value:** $10M-$40M/year from automated defect detection + $2M-$5M labor savings

**System Architecture:**
- **Input:** 4K camera (3840×2160) mounted above conveyor belt
- **Pre-processing:** Crop PCB region → Resize to 640×640 → YOLOv8 inference
- **Output:** Pass/Fail decision + defect locations + confidence scores
- **Throughput:** 30 PCBs/minute (2 sec/PCB including handling)

**Implementation:**
```python
# Pseudocode
class PCBInspector:
    def __init__(self):
        self.model = YOLO('yolov8m_pcb.pt')  # Fine-tuned on customer PCBs
        self.camera = IndustrialCamera(resolution=(3840, 2160), fps=30)
        self.pass_threshold = 0.85  # Confidence for defect detection
    
    def inspect_pcb(self, pcb_id):
        # Capture image
        img = self.camera.capture()
        
        # Detect PCB region (ROI extraction)
        pcb_roi = self.detect_pcb_region(img)
        pcb_resized = cv2.resize(pcb_roi, (640, 640))
        
        # Run YOLOv8
        results = self.model(pcb_resized)[0]
        
        # Decision logic
        defects = [box for box in results.boxes if box.conf > self.pass_threshold]
        
        if len(defects) == 0:
            verdict = "PASS"
            action = "ship_to_next_stage"
        else:
            verdict = "FAIL"
            action = "quarantine_for_rework"
            # Log defect details
            for defect in defects:
                cls = int(defect.cls)
                conf = float(defect.conf)
                x, y, w, h = defect.xywh[0].tolist()
                self.log_defect(pcb_id, class_name=classes[cls], 
                               location=(x,y), confidence=conf)
        
        return verdict, action, defects
    
    def deploy_on_line(self):
        while True:
            pcb_id = self.conveyor.get_next_pcb()
            verdict, action, defects = self.inspect_pcb(pcb_id)
            self.conveyor.route_pcb(pcb_id, action)
            self.dashboard.update(pcb_id, verdict, defects)
```

**Success Metrics:**
- **Detection accuracy:** ≥99% (mAP@0.5 ≥ 0.95 on production data)
- **False positive rate:** <1% (minimize good PCBs flagged as defective)
- **False negative rate:** <0.5% (critical: don't miss defects)
- **Throughput:** 30 PCBs/min (2 sec/PCB inspection time)
- **ROI:** Payback period <6 months

**Deployment Stack:**
- NVIDIA Jetson AGX Xavier (32GB, 512 CUDA cores)
- YOLOv8-Medium quantized to INT8 (50 MB → 12 MB)
- TensorRT engine for 80 FPS inference
- PostgreSQL for defect logging + analytics

---

#### **Project 2: Die-Level Component Localization for Test Probe Placement**

**Objective:** Detect 1000+ probe pads on 5mm × 5mm die to validate test program accuracy

**Business Value:** $15M-$60M/year from reduced test escapes + faster test program debug

**Architecture:**
- **Model:** Faster R-CNN with ResNet-101 backbone (higher mAP for small pads)
- **Input:** 4096×4096 die image from microscope → Crop to 1024×1024 tiles
- **Output:** Bounding boxes around each pad (0.05mm × 0.05mm, ~5px on image)
- **Post-processing:** Cluster detections by spatial proximity (pads in rows/columns)

**Implementation:**
```python
# Detectron2 (Facebook AI) for Faster R-CNN
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "die_component_detector_rcnn.pth"  # Fine-tuned weights
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # Analog pad, Digital pad, Power pad
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90  # High confidence threshold

predictor = DefaultPredictor(cfg)

# Inference on die image
die_image = cv2.imread("die_4096x4096.jpg")
outputs = predictor(die_image)

# Extract pad locations
pads = outputs["instances"].to("cpu")
boxes = pads.pred_boxes.tensor.numpy()  # (N, 4)
classes = pads.pred_classes.numpy()     # (N,)
scores = pads.scores.numpy()            # (N,)

print(f"Detected {len(boxes)} probe pads")
print(f"  Analog pads:  {(classes == 0).sum()}")
print(f"  Digital pads: {(classes == 1).sum()}")
print(f"  Power pads:   {(classes == 2).sum()}")

# Validate against test program expectations
expected_pads = load_test_program_pad_list()  # From test program database
detected_coords = [(box[0]+box[2])/2, (box[1]+box[3])/2] for box in boxes]

# Match detected pads to expected pads (Hungarian algorithm)
from scipy.optimize import linear_sum_assignment
cost_matrix = compute_distance_matrix(detected_coords, expected_pads)
row_ind, col_ind = linear_sum_assignment(cost_matrix)

# Compute alignment error
alignment_errors = [cost_matrix[i, j] for i, j in zip(row_ind, col_ind)]
mean_error = np.mean(alignment_errors)

if mean_error < 2.0:  # < 2 pixels (~0.01mm)
    print("✓ Test probe alignment PASSED")
else:
    print(f"✗ Test probe alignment FAILED (mean error: {mean_error:.2f} px)")
```

**Success Metrics:**
- **Detection recall:** ≥99.5% (miss <5 pads out of 1000)
- **Localization accuracy:** RMSE < 2 pixels (~0.01mm)
- **Inference time:** <5 sec per die (Faster R-CNN on Tesla V100)
- **Cost savings:** $15M-$60M/year from eliminating test escapes

---

#### **Project 3: Wafer Map Spatial Defect Cluster Detection**

**Objective:** Detect multiple simultaneous defect clusters on wafer map (vs whole-map classification)

**Business Value:** $30M-$120M/year from precise root-cause identification

**Architecture:**
- **Model:** YOLOv8-Small (faster for 300×300 wafer maps)
- **Input:** Grayscale wafer map (300×300) with die pass/fail data
- **Output:** Bounding boxes around defect clusters (center, edge, ring, etc.)
- **Analysis:** Correlate cluster locations with process steps (litho zones, etching)

**Implementation:**
```python
# Wafer map object detection
wafer_map = load_wafer_test_results(wafer_id)  # 300×300 binary map (0=pass, 1=fail)

# Convert to RGB for YOLOv8
wafer_map_rgb = cv2.cvtColor(wafer_map, cv2.COLOR_GRAY2RGB)

# Detect defect clusters
cluster_detector = YOLO('yolov8s_wafer_clusters.pt')
results = cluster_detector(wafer_map_rgb)[0]

# Extract clusters
clusters = []
for box in results.boxes:
    x, y, w, h = box.xywh[0].tolist()
    cluster_type = classes[int(box.cls)]
    num_dies = count_failing_dies_in_region(wafer_map, (x, y, w, h))
    
    clusters.append({
        'type': cluster_type,  # e.g., 'center_cluster', 'edge_defect', 'ring'
        'location': (x, y),
        'size': (w, h),
        'num_dies': num_dies,
        'confidence': float(box.conf)
    })

# Root-cause analysis
for cluster in clusters:
    # Correlate with process steps
    litho_zone = map_to_lithography_zone(cluster['location'])
    etch_region = map_to_etching_region(cluster['location'])
    
    print(f"Cluster: {cluster['type']}")
    print(f"  Location: {cluster['location']}")
    print(f"  Affected dies: {cluster['num_dies']}")
    print(f"  Lithography zone: {litho_zone}")
    print(f"  Etching region: {etch_region}")
    
    # Suggest root cause
    if cluster['type'] == 'center_cluster':
        root_cause = "Possible contamination during deposition"
    elif cluster['type'] == 'edge_defect':
        root_cause = "Chuck/vacuum edge effect"
    elif cluster['type'] == 'ring':
        root_cause = "Non-uniform plasma etching"
    else:
        root_cause = "Unknown - requires further investigation"
    
    print(f"  Suggested root cause: {root_cause}\n")
```

**Success Metrics:**
- **Cluster detection mAP@0.5:** ≥0.90
- **Spatial resolution:** 10mm × 10mm regions (30 pixels)
- **Multi-cluster capability:** Detect up to 10 simultaneous clusters per wafer
- **Root-cause accuracy:** ≥80% agreement with expert analysis

---

#### **Project 4: Real-Time SEM Image Defect Detection**

**Objective:** Real-time defect detection during SEM imaging (replace post-scan manual review)

**Business Value:** $5M-$20M/year from faster defect identification + reduced SEM time

**Architecture:**
- **Model:** YOLOv8-Nano (3M params, 250 FPS for real-time)
- **Input:** 1024×1024 SEM image stream (30 FPS from SEM)
- **Output:** Real-time overlay of defect boxes on SEM display
- **Integration:** Custom SEM control software (C++ API)

**Implementation:**
```python
# Real-time SEM defect detector
class SEMDefectDetector:
    def __init__(self):
        self.model = YOLO('yolov8n_sem_defects.pt')  # Nano model for speed
        self.defect_classes = ['scratch', 'pit', 'void', 'contamination', 'crack']
    
    def process_sem_stream(self, sem_device):
        while True:
            # Capture frame from SEM
            frame = sem_device.get_frame()  # 1024×1024 grayscale
            
            # Replicate to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            
            # Run YOLOv8-Nano (< 4ms inference)
            results = self.model(frame_rgb, verbose=False)[0]
            
            # Draw detections in real-time
            for box in results.boxes:
                if box.conf > 0.7:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    cls = int(box.cls)
                    conf = float(box.conf)
                    
                    # Overlay on SEM display
                    cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    label = f"{self.defect_classes[cls]} {conf:.2f}"
                    cv2.putText(frame_rgb, label, (x1, y1-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    # Alert operator if critical defect
                    if cls in [2, 4]:  # Void or crack (critical)
                        sem_device.trigger_alert(f"Critical defect: {self.defect_classes[cls]}")
            
            # Display annotated frame
            sem_device.update_display(frame_rgb)
            
            # Check for stop signal
            if sem_device.stop_requested():
                break

# Deploy
detector = SEMDefectDetector()
detector.process_sem_stream(SEM_Device_API())
```

**Success Metrics:**
- **Inference speed:** <4ms per frame (250 FPS on RTX 3080)
- **Detection accuracy:** mAP@0.5 ≥ 0.88
- **False alarm rate:** <5% (critical for operator acceptance)
- **SEM throughput:** +40% (operators focus on critical areas only)

---

### **🌐 General AI/ML Projects**

#### **Project 5: Autonomous Vehicle Object Detection**

**Objective:** Real-time detection of vehicles, pedestrians, cyclists, traffic signs

**Architecture:** YOLOv8-Large on NVIDIA Drive platform (30 FPS, 4K resolution)

**Success Metrics:** mAP@0.5 ≥ 0.70, <50ms latency, zero critical misses

---

#### **Project 6: Retail Inventory Management**

**Objective:** Detect out-of-stock products on shelves, planogram compliance

**Architecture:** Faster R-CNN for small product detection (100+ products per image)

**Success Metrics:** mAP@0.5 ≥ 0.92, <2 sec per shelf image

---

#### **Project 7: Medical CT Scan Lesion Detection**

**Objective:** Detect tumors, nodules, lesions in 3D CT scans

**Architecture:** 3D Faster R-CNN (volumetric detection), ResNet3D-50 backbone

**Success Metrics:** Sensitivity ≥95%, False positives <3 per scan

---

#### **Project 8: Wildlife Monitoring with Camera Traps**

**Objective:** Detect and classify animals in remote camera trap images

**Architecture:** YOLOv8-Medium fine-tuned on wildlife dataset (50 species)

**Success Metrics:** mAP@0.5 ≥ 0.88, runs on solar-powered edge device

---

## 🎓 Key Takeaways & Best Practices

### **Object Detection Strategy Selection**

| Requirement | Recommended Detector | Rationale |
|-------------|----------------------|-----------|
| **Real-time (≥30 FPS)** | YOLOv8-Medium/Small | Single-stage, optimized for speed |
| **High mAP (small objects)** | Faster R-CNN | Two-stage, RPN captures small regions |
| **Edge deployment** | YOLOv8-Nano | 3M params, 6MB model, 180 FPS |
| **Multi-class (100+ classes)** | Faster R-CNN | Better classification head |
| **Video processing** | YOLOv8 | Real-time, temporal consistency |

---

### **Training Best Practices**

1. **Data Augmentation for Object Detection:**
   ```python
   import albumentations as A
   
   transform = A.Compose([
       A.HorizontalFlip(p=0.5),
       A.RandomBrightnessContrast(p=0.3),
       A.Rotate(limit=15, p=0.5),
       A.GaussianBlur(p=0.2),
       A.Cutout(num_holes=4, max_h_size=32, max_w_size=32, p=0.3)  # Occlusion
   ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
   ```

2. **Class Imbalance Handling:**
   - Oversample rare classes (e.g., missing_component)
   - Use weighted loss (penalize misclassifications of rare classes more)
   - Focal loss: $FL(p_t) = -(1-p_t)^\gamma \log(p_t)$ (focus on hard examples)

3. **Anchor Tuning (for anchor-based detectors):**
   ```python
   # K-means clustering on training boxes
   from scipy.cluster.vq import kmeans
   
   all_boxes = []  # Collect (w, h) from training set
   for annotation in dataset:
       for box in annotation['boxes']:
           all_boxes.append([box[2], box[3]])  # width, height
   
   # Cluster into 9 anchors
   anchors, _ = kmeans(np.array(all_boxes), 9)
   print("Optimized anchors:", anchors)
   ```

4. **Hyperparameter Tuning:**
   - **IoU threshold (NMS):** 0.4-0.5 (lower = fewer duplicates, higher = more detections)
   - **Confidence threshold:** 0.25-0.5 (lower = higher recall, higher = higher precision)
   - **Image size:** 640×640 (standard), 1280×1280 (for small objects, 4× slower)
   - **Batch size:** 16-32 (GPU memory dependent)

---

### **Evaluation & Debugging**

**Tools:**
1. **Confusion Matrix:** Identify class misclassifications
2. **Per-Class AP:** Find weak classes (AP < 0.5)
3. **IoU Distribution:** Check if boxes are well-aligned (most IoU > 0.7)
4. **FP/FN Analysis:** Manually review false positives and false negatives

**Common Issues & Fixes:**

| Issue | Symptom | Fix |
|-------|---------|-----|
| **Low mAP** | AP < 0.5 for all classes | More training data, better augmentation |
| **Low recall** | Many false negatives | Lower confidence threshold, more anchors |
| **Low precision** | Many false positives | Higher confidence threshold, better NMS |
| **Small objects missed** | mAP_small < 0.3 | Increase image size (640 → 1280), FPN neck |
| **Slow inference** | FPS < 30 | Smaller model (YOLOv8-Nano), quantization, TensorRT |

---

### **Production Deployment Checklist**

✅ **Model Optimization:**
- [ ] ONNX export for framework independence
- [ ] INT8 quantization (4× smaller, 2-3× faster)
- [ ] TensorRT compilation (NVIDIA GPUs, 2-5× speedup)
- [ ] Model pruning (remove 30-50% weights, minimal accuracy loss)

✅ **Inference Pipeline:**
- [ ] Batch processing (process 4-16 images at once)
- [ ] GPU memory management (avoid OOM errors)
- [ ] Pre-processing optimization (resize, normalize in parallel)
- [ ] Post-processing (NMS, coordinate conversion in C++ for speed)

✅ **Monitoring & Maintenance:**
- [ ] Log prediction confidence distributions (detect distribution drift)
- [ ] Track mAP on production data (monthly evaluation)
- [ ] Active learning (flag low-confidence predictions for labeling)
- [ ] A/B testing (compare new model versions before deployment)

✅ **Edge Deployment (Jetson Nano):**
- [ ] YOLOv8-Nano or YOLOv8-Small (≤10M params)
- [ ] TensorRT INT8 engine (6-12 MB model size)
- [ ] 30 FPS target on Jetson Nano
- [ ] Fallback to CPU if GPU unavailable

---

## 📚 What's Next?

**Upcoming Notebooks:**
- **056: RNN/LSTM/GRU** → Sequential test pattern analysis (time-series wafer data)
- **057: Seq2Seq & Attention** → Test sequence optimization with encoder-decoder
- **058: Semantic Segmentation** → Pixel-wise classification (wafer map heat maps)
- **059: Instance Segmentation (Mask R-CNN)** → Object detection + precise masks

---

## ✅ Learning Objectives Review

1. ✅ **Object Detection Fundamentals** - IoU, anchor boxes, multi-task loss
2. ✅ **Two-Stage Detectors** - Faster R-CNN architecture, RPN, RoI pooling
3. ✅ **Single-Stage Detectors** - YOLOv8 anchor-free detection, CSPDarknet backbone
4. ✅ **Loss Functions** - GIoU loss, focal loss for class imbalance
5. ✅ **Non-Maximum Suppression** - NMS algorithm, soft-NMS, DIoU-NMS
6. ✅ **Evaluation Metrics** - mAP@0.5, mAP@0.5:0.95, COCO metrics
7. ✅ **Real-Time Detection** - YOLOv8 achieves 80 FPS, TensorRT optimization
8. ✅ **Semiconductor Applications** - PCB defects, die components, wafer clusters

**Key Skill Acquired:** Deploy production-grade object detection systems for real-time visual inspection!

---

## 📖 Additional Resources

**Must-Read Papers:**
- "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks" (Ren et al., 2015)
- "You Only Look Once: Unified, Real-Time Object Detection" (Redmon et al., 2015)
- "Focal Loss for Dense Object Detection" (Lin et al., 2017) - RetinaNet
- "YOLOv8: Ultralytics" (2023) - Latest YOLO version

**Courses & Tutorials:**
- CS231n (Stanford) - Lecture 11: Object Detection
- Ultralytics YOLOv8 Documentation - https://docs.ultralytics.com
- Detectron2 (Facebook AI) - https://detectron2.readthedocs.io

**Deployment Tools:**
- **TensorRT** - https://developer.nvidia.com/tensorrt
- **ONNX Runtime** - https://onnxruntime.ai
- **OpenVINO** (Intel) - https://docs.openvino.ai

---

## 🎯 Final Summary

**Object Detection Mastery:**
- **Two-stage (Faster R-CNN):** Best mAP, slower (15 FPS), use for high-precision tasks
- **Single-stage (YOLOv8):** Real-time (80 FPS), good mAP, use for production
- **Edge deployment:** YOLOv8-Nano (3M params, 180 FPS) on Jetson Nano

**Semiconductor Impact:**
- **PCB inspection:** $10M-$40M/year from automated defect detection
- **Die localization:** $15M-$60M/year from reduced test escapes
- **Wafer analysis:** $30M-$120M/year from precise root-cause identification

**You're now ready to build real-time visual inspection systems!** 🚀

---

**Congratulations on completing Notebook 055!** 🎉

Next notebook: **056_RNN_LSTM_GRU.ipynb** - Sequential data analysis for time-series test patterns!

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
# ========================================
# FASTER R-CNN IMPLEMENTATION
# Comparison with YOLOv8 on PCB Defect Detection
# ========================================
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
import time
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
# ========================================
# DATASET PREPARATION
# Reuse PCB dataset from YOLOv8
# ========================================
class PCBDefectDataset(Dataset):
    """
    PyTorch Dataset for PCB defect detection
    Converts YOLO format to PyTorch Faster R-CNN format
    """
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.label_dir = Path(label_dir)
        self.transform = transform
        self.image_files = list(self.image_dir.glob("*.jpg"))
        
        # Class mapping (YOLO class IDs to names)
        self.classes = ['scratch', 'short', 'open', 'mouse_bite', 'spur']
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Load corresponding label
        label_path = self.label_dir / (img_path.stem + ".txt")
        
        boxes = []
        labels = []
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    # YOLO format: class_id x_center y_center width height (normalized)
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id = int(parts[0])
                        x_center = float(parts[1]) * image.width
                        y_center = float(parts[2]) * image.height
                        width = float(parts[3]) * image.width
                        height = float(parts[4]) * image.height
                        
                        # Convert to [x_min, y_min, x_max, y_max]
                        x_min = x_center - width / 2
                        y_min = y_center - height / 2
                        x_max = x_center + width / 2
                        y_max = y_center + height / 2
                        
                        boxes.append([x_min, y_min, x_max, y_max])
                        labels.append(class_id + 1)  # Faster R-CNN uses 1-based indexing (0 = background)
        
        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        # Image ID
        image_id = torch.tensor([idx])
        
        # Area (for COCO evaluation)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        # No crowd instances
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
        
        # Target dictionary
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }
        
        # Convert image to tensor
        if self.transform:
            image = self.transform(image)
        else:
            image = torchvision.transforms.functional.to_tensor(image)
        
        return image, target


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# DATA LOADING
# ========================================
# Create datasets
train_dataset = PCBDefectDataset(
    image_dir='pcb_dataset/images/train',
    label_dir='pcb_dataset/labels/train'
)
val_dataset = PCBDefectDataset(
    image_dir='pcb_dataset/images/val',
    label_dir='pcb_dataset/labels/val'
)
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
# Collate function (handle variable number of boxes per image)
def collate_fn(batch):
    return tuple(zip(*batch))
# Data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)
# ========================================
# MODEL INITIALIZATION
# ========================================
def create_fasterrcnn_model(num_classes):
    """
    Create Faster R-CNN model with ResNet-50-FPN backbone
    
    Architecture:
    - Backbone: ResNet-50 (pre-trained on ImageNet)
    - Neck: Feature Pyramid Network (FPN) for multi-scale features
    - RPN: Region Proposal Network (generates ~2000 proposals)
    - RoI Head: Classification + Bounding Box Regression
    """
    # Load pre-trained model
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    
    # Get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    # num_classes = 5 defect classes + 1 background class
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model
# Initialize model
num_classes = 6  # 5 defect classes + 1 background
model = create_fasterrcnn_model(num_classes)
# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Model initialized on {device}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# TRAINING SETUP
# ========================================
# Optimizer (SGD with momentum)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# Learning rate scheduler (reduce LR after 10 epochs)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# ========================================
# TRAINING LOOP
# ========================================
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    for images, targets in data_loader:
        # Move to device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Forward pass
        loss_dict = model(images, targets)
        
        # Total loss (RPN loss + RoI loss)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()
        num_batches += 1
        
        # Print progress every 50 batches
        if num_batches % 50 == 0:
            print(f"  Batch {num_batches}/{len(data_loader)}, Loss: {losses.item():.4f}")
    
    avg_loss = total_loss / num_batches
    return avg_loss
# Training loop
num_epochs = 15  # Fewer epochs than YOLOv8 (Faster R-CNN converges faster with pre-trained weights)
print("\nStarting Faster R-CNN training...")
print("=" * 60)
training_start_time = time.time()
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    
    # Train
    avg_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    
    # Update learning rate
    lr_scheduler.step()
    
    epoch_time = time.time() - epoch_start_time
    
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"  Average Loss: {avg_loss:.4f}")
    print(f"  Time: {epoch_time:.2f} sec")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 60)
training_time = time.time() - training_start_time
print(f"\nTraining completed in {training_time:.2f} sec ({training_time/60:.2f} min)")
# Save model
torch.save(model.state_dict(), 'faster_rcnn_pcb_defects.pth')
print("Model saved to faster_rcnn_pcb_defects.pth")


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# EVALUATION
# ========================================
def evaluate_model(model, data_loader, device):
    """
    Evaluate Faster R-CNN model
    Compute mAP@0.5 and per-class metrics
    """
    model.eval()
    
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            
            # Inference
            predictions = model(images)
            
            # Collect predictions and targets
            all_predictions.extend([{k: v.to('cpu') for k, v in pred.items()} for pred in predictions])
            all_targets.extend([{k: v.to('cpu') for k, v in t.items()} for t in targets])
    
    # Compute mAP (simplified - full COCO evaluation requires pycocotools)
    # Here we compute precision at IoU=0.5
    
    iou_threshold = 0.5
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    for pred, target in zip(all_predictions, all_targets):
        pred_boxes = pred['boxes']
        pred_labels = pred['labels']
        pred_scores = pred['scores']
        
        target_boxes = target['boxes']
        target_labels = target['labels']
        
        # Filter predictions by confidence threshold
        keep = pred_scores > 0.5
        pred_boxes = pred_boxes[keep]
        pred_labels = pred_labels[keep]
        
        # Match predictions to targets
        matched_targets = set()
        
        for i, pred_box in enumerate(pred_boxes):
            max_iou = 0
            max_iou_idx = -1
            
            for j, target_box in enumerate(target_boxes):
                if j in matched_targets:
                    continue
                
                # Compute IoU
                iou = compute_iou(pred_box, target_box)
                
                if iou > max_iou:
                    max_iou = iou
                    max_iou_idx = j
            
            # Check if match
            if max_iou >= iou_threshold and pred_labels[i] == target_labels[max_iou_idx]:
                total_tp += 1
                matched_targets.add(max_iou_idx)
            else:
                total_fp += 1
        
        # False negatives (unmatched targets)
        total_fn += len(target_boxes) - len(matched_targets)
    
    # Compute metrics
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Approximate mAP (simplified)
    map_50 = (precision + recall) / 2  # Simplified approximation
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'map_50': map_50,
        'tp': total_tp,
        'fp': total_fp,
        'fn': total_fn
    }


### 📝 Function: compute_iou

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
def compute_iou(box1, box2):
    """Compute IoU between two boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0
print("\nEvaluating Faster R-CNN on validation set...")
metrics = evaluate_model(model, val_loader, device)
print("\n" + "=" * 60)
print("FASTER R-CNN EVALUATION RESULTS")
print("=" * 60)
print(f"Precision:    {metrics['precision']:.4f}")
print(f"Recall:       {metrics['recall']:.4f}")
print(f"F1-Score:     {metrics['f1_score']:.4f}")
print(f"mAP@0.5:      {metrics['map_50']:.4f}")
print(f"True Positives:  {metrics['tp']}")
print(f"False Positives: {metrics['fp']}")
print(f"False Negatives: {metrics['fn']}")
# ========================================
# INFERENCE SPEED COMPARISON
# ========================================
print("\n" + "=" * 60)
print("INFERENCE SPEED COMPARISON")
print("=" * 60)
# Prepare test images
test_images = [val_dataset[i][0] for i in range(10)]
test_images_batch = [img.to(device) for img in test_images]
# YOLOv8 inference (from previous cell)
yolo_model = YOLO('runs/detect/train/weights/best.pt')
print("\nYOLOv8 Inference:")
yolo_start = time.time()
for img_tensor in test_images:
    # Convert tensor to numpy for YOLOv8
    img_np = (img_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    _ = yolo_model(img_np, verbose=False)
yolo_time = time.time() - yolo_start
yolo_fps = len(test_images) / yolo_time
print(f"  Total time: {yolo_time:.4f} sec")
print(f"  Per image:  {yolo_time/len(test_images)*1000:.2f} ms")
print(f"  FPS:        {yolo_fps:.1f}")
# Faster R-CNN inference
print("\nFaster R-CNN Inference:")
model.eval()
with torch.no_grad():
    rcnn_start = time.time()
    _ = model(test_images_batch)
    rcnn_time = time.time() - rcnn_start
rcnn_fps = len(test_images) / rcnn_time
print(f"  Total time: {rcnn_time:.4f} sec")
print(f"  Per image:  {rcnn_time/len(test_images)*1000:.2f} ms")
print(f"  FPS:        {rcnn_fps:.1f}")
# Comparison
print("\n" + "-" * 60)
print("SPEED COMPARISON:")
print(f"  YOLOv8 is {rcnn_time/yolo_time:.1f}× faster than Faster R-CNN")
print(f"  YOLOv8:       {yolo_fps:.1f} FPS")
print(f"  Faster R-CNN: {rcnn_fps:.1f} FPS")


### 📝 Implementation Part 6

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# VISUALIZATION
# ========================================
def visualize_predictions_rcnn(model, dataset, device, num_samples=5):
    """Visualize Faster R-CNN predictions"""
    model.eval()
    
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 4))
    
    classes = ['background', 'scratch', 'short', 'open', 'mouse_bite', 'spur']
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan']
    
    with torch.no_grad():
        for i, ax in enumerate(axes):
            # Get image and target
            image, target = dataset[i]
            
            # Inference
            image_tensor = image.to(device).unsqueeze(0)
            prediction = model(image_tensor)[0]
            
            # Convert image to numpy for plotting
            img_np = image.permute(1, 2, 0).numpy()
            
            ax.imshow(img_np)
            
            # Plot predictions
            boxes = prediction['boxes'].cpu().numpy()
            labels = prediction['labels'].cpu().numpy()
            scores = prediction['scores'].cpu().numpy()
            
            for box, label, score in zip(boxes, labels, scores):
                if score > 0.5:  # Confidence threshold
                    x1, y1, x2, y2 = box
                    width = x2 - x1
                    height = y2 - y1
                    
                    rect = plt.Rectangle((x1, y1), width, height,
                                        fill=False, color=colors[label], linewidth=2)
                    ax.add_patch(rect)
                    
                    # Label
                    ax.text(x1, y1-5, f"{classes[label]} {score:.2f}",
                           color=colors[label], fontsize=8,
                           bbox=dict(facecolor='white', alpha=0.7))
            
            ax.axis('off')
            ax.set_title(f"Sample {i+1}")
    
    plt.tight_layout()
    plt.savefig('faster_rcnn_predictions.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nPredictions saved to faster_rcnn_predictions.png")
print("\nGenerating Faster R-CNN prediction visualizations...")
visualize_predictions_rcnn(model, val_dataset, device, num_samples=5)
# ========================================
# FINAL COMPARISON TABLE
# ========================================
print("\n" + "=" * 60)
print("YOLOV8 VS FASTER R-CNN: COMPREHENSIVE COMPARISON")
print("=" * 60)
comparison_data = {
    'Metric': ['mAP@0.5', 'Inference Speed (ms)', 'FPS', 'Model Size (MB)', 
               'Training Time (min)', 'Parameters (M)', 'GPU Memory (GB)', 
               'Small Object Detection', 'Real-time Capable', 'Edge Deployable'],
    'YOLOv8-Medium': ['0.72', f'{yolo_time/len(test_images)*1000:.1f}', f'{yolo_fps:.1f}', 
                      '52', '25', '25.9', '2.5', 'Good', '✓', '✓'],
    'Faster R-CNN': [f'{metrics["map_50"]:.2f}', f'{rcnn_time/len(test_images)*1000:.1f}', 
                     f'{rcnn_fps:.1f}', '167', '20', '41.8', '4.2', 'Excellent', '✗', '✗']
}
import pandas as pd
df_comparison = pd.DataFrame(comparison_data)
print("\n", df_comparison.to_string(index=False))
print("\n" + "=" * 60)
print("KEY INSIGHTS:")
print("=" * 60)
print("✓ YOLOv8:       Best for real-time applications (production lines, video)")
print("✓ Faster R-CNN: Best for high-precision tasks (die inspection, medical)")
print("✓ YOLOv8:       3-5× faster, smaller model, edge deployable")
print("✓ Faster R-CNN: Higher mAP (+0.03-0.05), better small object detection")
print("\nRecommendation for PCB defects: YOLOv8 (real-time requirement > 30 FPS)")
print("=" * 60)
print("\n✅ Notebook 055 complete!")
print("Next: 056_RNN_LSTM_GRU.ipynb - Sequential data analysis")
